<a href="https://colab.research.google.com/github/sheshrajkandel/news-sentiment-stock-price-prediction/blob/main/GenerateNewsSentimentScores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [176]:
import nltk
import pandas as pd
import datetime
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#from google.colab import drive
#drive.mount('/content/drive')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [178]:
input_file = "/content/drive/MyDrive/Dissertation/news_nabil.csv"
df_news = pd.read_csv(input_file)

df_news.head()

,Date,Title
0,5/22/2022,Details Regarding The Press Release Of Nabil B...
1,5/19/2022,Nabil Bank signs MoU with Office of Investment...
2,5/15/2022,Nabil Bank ties up with Payment Nepal for UPI ...
3,5/9/2022,Nabil Bank rewards ‘Shree Shambhu Prasad Poudy...
4,5/8/2022,Nabil Bank becomes first and only Bank in Nepa...


In [179]:
titles = df_news["Title"]
#print(titles)

###Generate Sentiment Analysis Score from News Headlines

In [180]:
output_file = "/content/drive/MyDrive/Dissertation/news_nabil_score.csv"

score_list = []
for title in titles:
  sid = SentimentIntensityAnalyzer()
  ss = sid.polarity_scores(title)
  #print(title)
  #print(ss['compound'])
  score_list.append(ss['compound'])

score_df = pd.DataFrame(score_list)
#print(score_df)

df_news['SA_Score'] = score_df[0]
df_news.to_csv(output_file, index = False)

In [181]:
df_news.head()

,Date,Title,SA_Score
0,5/22/2022,Details Regarding The Press Release Of Nabil B...,0.0000
1,5/19/2022,Nabil Bank signs MoU with Office of Investment...,0.3818
2,5/15/2022,Nabil Bank ties up with Payment Nepal for UPI ...,0.4019
3,5/9/2022,Nabil Bank rewards ‘Shree Shambhu Prasad Poudy...,0.4767
4,5/8/2022,Nabil Bank becomes first and only Bank in Nepa...,0.5574


###**Find the date gaps and fill the sentiment analysis scores with 0**

In [182]:
input_stock_data = "/content/drive/MyDrive/Dissertation/nabil_stock_data.csv"
df_stock_data = pd.read_csv(input_stock_data)

input_news_score = "/content/drive/MyDrive/Dissertation/news_nabil_score.csv"
df_news_score = pd.read_csv(input_news_score)

#Create a column SA_Score and set 0 as default value
df_stock_data['SA_Score'] = 0 

df_news_score['Date'] = pd.to_datetime(df_news_score['Date'], format='%m/%d/%Y').dt.date
df_stock_data['Date'] = pd.to_datetime(df_stock_data['Date'], format='%m/%d/%Y').dt.date
#df_news_score['SA_Score'].apply(lambda x: float(x))

In [184]:
df_news_score.head()

,Date,Title,SA_Score
0,2022-05-22,Details Regarding The Press Release Of Nabil B...,0.0000
1,2022-05-19,Nabil Bank signs MoU with Office of Investment...,0.3818
2,2022-05-15,Nabil Bank ties up with Payment Nepal for UPI ...,0.4019
3,2022-05-09,Nabil Bank rewards ‘Shree Shambhu Prasad Poudy...,0.4767
4,2022-05-08,Nabil Bank becomes first and only Bank in Nepa...,0.5574


In [185]:
df_stock_data.head()

,Date,Open,High,Low,Ltp,% Change,Qty,Turnover,SA_Score
0,2022-05-23,906,941,898,899,-0.77,"45,910.00","41,509,265.50",0
1,2022-05-20,901,915,900,906,-0.22,"25,594.00","23,127,532.00",0
2,2022-05-19,929,929,896,908,-0.66,"29,333.00","26,676,250.10",0
3,2022-05-18,915,918,905.3,914,0.44,"24,747.00","22,492,071.40",0
4,2022-05-17,930,935,908,910,-2.15,"26,873.00","24,558,509.30",0


###**Assign SA_Score of previous date to current date and prepare a final stock dataset with SA_Score**

In [186]:
def get_score(previous_date):
  score = 0.0 
  if(previous_date in df_news_score['Date'].values):
    temp_df = df_news_score.loc[df_news_score['Date'] == previous_date]
    score = temp_df.iloc[0]['SA_Score']
    
  return score

for index, row in df_stock_data.iterrows():
  if(index+1 == len(df_stock_data.index)):
    continue
  else:
    previous_date = df_stock_data.loc[index + 1, 'Date']
  #print(row['Date'], index)
  current_date = row['Date']
  
  score_counter = 0
  total_score = 0.0

  while(current_date > previous_date):
    sa_score = get_score(previous_date)
    #print('SA_Score: ', sa_score)
    if(sa_score != 0):
      total_score = sa_score + total_score
      #print('Total: ', total_score)
      score_counter = score_counter + 1
      #print('Average: ', score_counter)
    previous_date = previous_date + datetime.timedelta(days = 1)
    if(score_counter > 0):
      final_score = total_score / score_counter      
    else:
      final_score = 0.0
    #print('Final Score: ', final_score)
  df_stock_data.loc[index, 'SA_Score'] = final_score

#df_stock_data

In [187]:
df_stock_data.head()

,Date,Open,High,Low,Ltp,% Change,Qty,Turnover,SA_Score
0,2022-05-23,906,941,898,899,-0.77,"45,910.00","41,509,265.50",0.0000
1,2022-05-20,901,915,900,906,-0.22,"25,594.00","23,127,532.00",0.3818
2,2022-05-19,929,929,896,908,-0.66,"29,333.00","26,676,250.10",0.0000
3,2022-05-18,915,918,905.3,914,0.44,"24,747.00","22,492,071.40",0.0000
4,2022-05-17,930,935,908,910,-2.15,"26,873.00","24,558,509.30",0.4019


In [188]:
#Write stock data with SA_Score to a new csv file 
stock_data_score = "/content/drive/MyDrive/Dissertation/nabil_stock_data_score.csv"
df_stock_data.to_csv(stock_data_score, index = False)